# Objectifs de ce Notebook
En application du TD3, je vais explorer ce que l’on pourrait faire avec les ontologies côté arXive, sur le modèle de ce que propose HAL. (Il y a donc in fine l’idée de proposer pour arXive une ontologie sommaire pour le domaine cs.AI. Du coup, je ne créé pas ici d’ontologie dans protégé, je le ferai plus tard pour correspondre au projet fil rouge (PFR) sur arXive.)
Dans le cadre du PFR, on pourrait vérifier/compléter nos données avec des requêtes dans des endpoints HAL (par exemple rechercher un doc archive dans HAL, ou des auteurs) ou d’autres.
Je vais créer une ontologie simple, directement avec owlready2, puis je vais la peupler avec SPARQLWrapper.
Dans un 3e temps, je vais tenter d’explorer l’emploi du raisonneur, voire d’autres éléments.

## I. Création de l’ontologie dans PYTHON

### I. Création

In [1]:
from owlready2 import *
from SPARQLWrapper import *
#from rdflib.namespace._FOAF import FOAF  # génère un warning

#il n'y a pas le logger standard (import logging, maintenant dans python) mais la doc propose :
#set_log_level(9)
set_log_level(0) #  pour désactiver

# Je vais modifier l'ontologie du TD, je la renomme. Elle sera basée sur FOAF puisque HAL l'utilise.
onto = get_ontology('https://cp.org/prepaFilRouge.owl#')
foaf = get_ontology('foaf.owl').load()
onto.imported_ontologies.append(foaf)

* Owlready2 * WARNING: DataProperty http://xmlns.com/foaf/0.1/nick belongs to more than one entity types: [owl.ObjectProperty, owl.DatatypeProperty]; I'm trying to fix it...
* Owlready2 * WARNING: DataProperty http://xmlns.com/foaf/0.1/aimChatID belongs to more than one entity types: [owl.ObjectProperty, owl.InverseFunctionalProperty, owl.DatatypeProperty, foaf.nick]; I'm trying to fix it...
* Owlready2 * WARNING: DataProperty http://xmlns.com/foaf/0.1/icqChatID belongs to more than one entity types: [owl.ObjectProperty, owl.InverseFunctionalProperty, owl.DatatypeProperty, foaf.nick]; I'm trying to fix it...
* Owlready2 * WARNING: DataProperty http://xmlns.com/foaf/0.1/jabberID belongs to more than one entity types: [owl.ObjectProperty, owl.InverseFunctionalProperty, owl.DatatypeProperty]; I'm trying to fix it...
* Owlready2 * WARNING: DataProperty http://xmlns.com/foaf/0.1/mbox_sha1sum belongs to more than one entity types: [owl.ObjectProperty, owl.InverseFunctionalProperty, owl.Datat

In [2]:
with onto:

    class Test(Thing):pass

    class monFoafEcritPar(ObjectProperty):
        #domaine = [foaf.Document]
        #range = [foaf.Person]
        inverse = foaf.publications
    monFoafEcritPar.comment =["fonction inverse de foaf.publications"]

    class Personne(foaf.Person): pass
    Personne.comment = ["Définition de la classe Personne qui spécialise foaf.Person"]

    class halPerson(Personne >> foaf.Person): pass
    halPerson.comment = [" link to foaf.Person from HAL, not necessary HalID !"]

    class halIdPerson(Personne >> foaf.Person, FunctionalProperty): pass
    halIdPerson.comment = [" link to halId"]

    class fullName(Personne >> str,FunctionalProperty): pass
    fullName.comment = ["Remplace name, déjà pris par owlready2"]


    class Publication(foaf.Document): pass
    Publication.comment = ["Définition de la classe Publication qui spéciffie foaf.Document"]
    Publication.comment.append("in fine, comme dans HAL on devrait pouvoir décrire le type d'un document dans arXive : préprint, affiche, these, livre, rapport")

    class halDocument(Publication >> foaf.Document, FunctionalProperty): pass  # TODO test pb en fonction de la version du doc ?
    halDocument.comment = ["lien vers le depot HAL"]

    class arXiveDocument(Publication >> foaf.Document,FunctionalProperty ) : pass
    arXiveDocument.comment = ["lien vers le depot arXive"]

    class ecrit(foaf.publications):
        domaine = [Personne]
        range = [Publication]
    ecrit.comment = ["spécialisation de foaf.publications"]

    class ecritPar(monFoafEcritPar):
        domaine = [Publication]
        range = [Personne]
        #inverse = ecrit # pas nécessaire vu les régles données au dessus
    #TODO un comment

    class maPersonne(ObjectProperty):
        #domaine = [foaf.Person |foaf.Document]  # ce type de ligne, avec un ou n'est pas sauvegardé dans le fichier owl.
        range = [Personne]
    maPersonne.comment = ["lien vers Personne, pour le raisonneur"]

    class maPersonneFromPerson(maPersonne,FunctionalProperty):
        domaine = [foaf.Person]
    maPersonneFromPerson.comment = ["spécialisation de maPersonne pour les Person"]

    class maPersonneFromDocument(maPersonne):
        domaine = [foaf.Document]
    maPersonneFromDocument.comment = ["spécialisation de maPersonne pour les Document"]

    class maPublication(ObjectProperty):
        #domaine = [foaf.Person |foaf.Document] # ce type de ligne, avec un ou n'est pas sauvegardé dans le fichier owl.
        range = [Publication]
    maPublication.comment = ["lien vers Publication, pour le raisonneur"]

    class maPublicationFromPerson(maPublication):
        domaine = [foaf.Person]
    maPublicationFromPerson.comment = ["spécialisation de maPublication pour les Person"]

    class maPublicationFromDocument(maPublication,FunctionalProperty):
        domaine = [foaf.Document]
    maPublicationFromDocument.comment = ["spécialisation de maPublication pour les Document"]


    class Affiliation(foaf.Organization): pass
    Affiliation.comment = ["Organisation (au sens foaf) d'accueil d'un auteur de publication "]

    class label(Affiliation >> str,FunctionalProperty): pass # DataProperty):
    label.comment = [" Le prefLabel d'une organization"]

    class url(Affiliation >> str,FunctionalProperty): pass   # avant >> str
    url.comment = ["L'url de l'organisazion, presque foaf.homepage mais sans imposer un foaf.Document "]

    class status(Affiliation>>str, FunctionalProperty):pass #DataProperty
    status.comment = ["Indique l'état actuel de l'organization, typiquement valid ou old"]

    class affiliationType (Affiliation >>str, FunctionalProperty ):pass
    affiliationType.comment = ["Caractérise l'organisation : institution, comme une école, Laboratory ou Researchteam au sein d'un labo"]
    affiliationType.comment.append("Pour le moment je ne gère pas la relation de composition des organizations")

    AllDisjoint([Personne, Publication,Affiliation])

### I.2 Visualisation de l'ontologie dans python

In [3]:
# On peut explorer l'ontologie en python :
for i in foaf.classes():  #All classes
    print (i)
#for i in foaf.individuals():
#    print (i)
#properties(): #All properties
#object_properties(): #All object properties
#annotation_properties(): #All annotation properties
#disjoints(): #All pairwise disjoints (including pairwise distinct individuals and disjoint/distinct pairs)
#disjoint_classes()
#disjoint_properties() All pairwise disjoint properties (including disjoint pairs of properties)
#different_individuals() All pairwise distinct individuals (including distinct pairs of individuals)
#rules() All SWRL rules
#variables() All SWRL variables
#general_axioms() All general axioms

foaf.Agent
foaf.OnlineAccount
foaf.Document
foaf.Person
foaf.Image
foaf.Group
foaf.Organization


On peut également visualier plus spécifiquement une propriété

In [4]:
print (foaf.publications.domain,foaf.publications.range)

[foaf.Person] [foaf.Document]


### I.3 Vérification  de l'ontologie dans python
Test rapide du modèle avec un raisonneur.
J'ai un message d'erreur d'owlready2, je vais donc sauver l'ontologie et la tester depuis protégé.

In [5]:
onto.save("prepaPFR_1.owl")

Aucune erreur n'est détectée sur ce fichier.

## II. Peuplement de l'ontologie

Je recherche tous les auteurs de HAL qui se nomment YOLAINE BOURDA. (En faisant abstraction des majuscules.)

## II.1 import d’auteurs Hal, première manière IRI déréférencable

Je ne vais rien dupliquer et conserver les IRI qui sont souvent déréférencable dans HAL.

In [6]:
#from owlready2.sparql.endpoint import *
sparql = SPARQLWrapper("http://sparql.archives-ouvertes.fr/sparql")
sparql.setTimeout(60)
query = """prefix foaf: <http://xmlns.com/foaf/0.1/>
select distinct ?s
where {
 ?s a foaf:Person; foaf:familyName ?nomFamille.
 #OPTIONAL {?s <http://www.openarchives.org/ore/terms/isAggregatedBy> ?o}
 ?s foaf:firstName ?prenom.
 #?s foaf:name ?name.

 FILTER (UCASE( ?prenom)="YOLAINE").
 FILTER (UCASE( ?nomFamille)="BOURDA").
}
LIMIT 100
"""

In [7]:
def extractUriValues (jsonSrc):  # Fonction pour extraire les URI du résultat d'une requête
    tabl = jsonSrc["results"]["bindings"]
    newResult = []
    for t in tabl:
        for k,v in t.items():
            if 'type' in v:
                if v['type']=='uri':
                    newResult.append(v['value'])
    return newResult

In [8]:
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
tabFOAF = extractUriValues(results)
print(tabFOAF)

['https://data.archives-ouvertes.fr/author/46799', 'https://data.archives-ouvertes.fr/author/270747', 'https://data.archives-ouvertes.fr/author/yolaine-bourda', 'https://data.archives-ouvertes.fr/author/1420714', 'https://data.archives-ouvertes.fr/author/4635', 'https://data.archives-ouvertes.fr/author/1295368']


In [9]:
# Test manuel sur un cas
#px = foaf.Person()
#px.set_iri('https://data.archives-ouvertes.fr/author/46799')
#t= Personne(name=None,namespace=onto,halPerson = [px])


Je vérifie que ces individus sont bien dans l'ontologie

In [10]:
with onto:
    for i in tabFOAF:
        p = foaf.Person(namespace=onto)
        p.set_iri(i)
        # p.isDefinedBy = ["http://sparql.archives-ouvertes.fr/"]
        pers = Personne(halPerson = [p])
        p.maPersonneFromPerson = pers # [pers]

In [11]:
for i in onto.individuals():
    print (i,i.is_a)
    for prop in i.get_properties():
        for value in prop[i]:
            print(".%s == %s" % (prop.python_name, value))

author.46799 [foaf.Person]
.maPersonneFromPerson == prepaFilRouge.personne1
prepaFilRouge.personne1 [prepaFilRouge.Personne]
.halPerson == author.46799
author.270747 [foaf.Person]
.maPersonneFromPerson == prepaFilRouge.personne2
prepaFilRouge.personne2 [prepaFilRouge.Personne]
.halPerson == author.270747
author.yolaine-bourda [foaf.Person]
.maPersonneFromPerson == prepaFilRouge.personne3
prepaFilRouge.personne3 [prepaFilRouge.Personne]
.halPerson == author.yolaine-bourda
author.1420714 [foaf.Person]
.maPersonneFromPerson == prepaFilRouge.personne4
prepaFilRouge.personne4 [prepaFilRouge.Personne]
.halPerson == author.1420714
author.4635 [foaf.Person]
.maPersonneFromPerson == prepaFilRouge.personne5
prepaFilRouge.personne5 [prepaFilRouge.Personne]
.halPerson == author.4635
author.1295368 [foaf.Person]
.maPersonneFromPerson == prepaFilRouge.personne6
prepaFilRouge.personne6 [prepaFilRouge.Personne]
.halPerson == author.1295368


In [12]:
for i in onto.individuals():
    destroy_entity(i)

### II.2 import d'auteurs Hal, deuxième manière : duplication des données
Avec la requête, je vais préciser les éléments que je souhaite utiliser pour peupler l'ontologie

In [13]:
query="""prefix foaf: <http://xmlns.com/foaf/0.1/>
select distinct ?s ?name ?idHal
where {
 ?s a foaf:Person; foaf:familyName ?nomFamille.
 OPTIONAL {?s <http://www.openarchives.org/ore/terms/isAggregatedBy> ?idHal}
 ?s foaf:firstName ?prenom.
 ?s foaf:name ?name.

 FILTER (UCASE( ?prenom)="YOLAINE").
 FILTER (UCASE( ?nomFamille)="BOURDA").
}
LIMIT 100
"""

In [14]:
sparql.setTimeout(60)
sparql.setQuery(query)
sparql.setReturnFormat(CSV)
results = sparql.query().convert()

In [15]:
print (results)

b'"s","name","idHal"\n"https://data.archives-ouvertes.fr/author/46799","Yolaine Bourda","https://data.archives-ouvertes.fr/author/yolaine-bourda"\n"https://data.archives-ouvertes.fr/author/270747","Yolaine Bourda","https://data.archives-ouvertes.fr/author/yolaine-bourda"\n"https://data.archives-ouvertes.fr/author/yolaine-bourda","Yolaine Bourda",\n"https://data.archives-ouvertes.fr/author/1420714","Yolaine Bourda",\n"https://data.archives-ouvertes.fr/author/4635","Yolaine Bourda","https://data.archives-ouvertes.fr/author/yolaine-bourda"\n"https://data.archives-ouvertes.fr/author/1295368","Yolaine Bourda","https://data.archives-ouvertes.fr/author/yolaine-bourda"\n'


Je peux maintenant peupler l'ontologie, mais je dois procéder en deux temps car lorsque je créé la première foaf.Person, l'idHal (une foaf.Person également) n'existe pas encore.


In [16]:
tmpResult = results.decode().split('\n')[1:-1]# je n'utilise pas la ligne de titre ni la dernière (vide)
for l in tmpResult:
    #print(l)
    s,name,idHal=l.split(',')
    print(s,name,idHal)
    s = s.strip('"')
    name = name.strip('"')
    #print(s,name,idHal)
    p = foaf.Person(namespace=onto)
    p.set_iri(s)
    #p.isDefinedBy = ["http://sparql.archives-ouvertes.fr/"]
    pers = Personne(name=None,namespace=onto,halPerson = [p])
    pers.fullName = name
    p.maPersonneFromPerson = pers # [pers]

"https://data.archives-ouvertes.fr/author/46799" "Yolaine Bourda" "https://data.archives-ouvertes.fr/author/yolaine-bourda"
"https://data.archives-ouvertes.fr/author/270747" "Yolaine Bourda" "https://data.archives-ouvertes.fr/author/yolaine-bourda"
"https://data.archives-ouvertes.fr/author/yolaine-bourda" "Yolaine Bourda" 
"https://data.archives-ouvertes.fr/author/1420714" "Yolaine Bourda" 
"https://data.archives-ouvertes.fr/author/4635" "Yolaine Bourda" "https://data.archives-ouvertes.fr/author/yolaine-bourda"
"https://data.archives-ouvertes.fr/author/1295368" "Yolaine Bourda" "https://data.archives-ouvertes.fr/author/yolaine-bourda"


In [17]:
trueIdHal = "https://data.archives-ouvertes.fr/author/yolaine-bourda"

In [18]:
# Toutes les foaf.Person sont créées je peux stocker l'idHal (qui ici sera toujours le même).
trueIdHalPerson = onto.search_one(type = foaf.Person, iri = trueIdHal)
print(trueIdHalPerson,trueIdHalPerson.is_a)

author.yolaine-bourda [foaf.Person]


In [19]:
def myStrip(var:str):
    if var is None:
        return None
    var = var.strip('"')
    if len(var)== 0:
        return None
    return var

for l in tmpResult:
    #print(l)
    s,name,idHal=l.split(',')
    #print(s,name,idHal)
    s = myStrip(s)
    name = myStrip(name)
    idHal = myStrip(idHal)
    #print(s,name,idHal)
    if idHal is not None:
        #print(s, idHal)
        auteur = onto.search_one(type = foaf.Person, iri = s)
        pers = onto.search_one(type = onto.Personne,halPerson = [auteur])
        pers.halIdPerson = trueIdHalPerson


In [20]:
onto.save("prepaPFR_2.owl")  # Pour vérification dans Protégé

### II.3 import des structures HAL
Je vais importer dans l'ontologie les structures de mes personnes.

In [21]:
query="""PREFIX hal:<http://data.archives-ouvertes.fr/schema/>
PREFIX skos:<http://www.w3.org/2004/02/skos/core#>
PREFIX vcard:<http://www.w3.org/2006/vcard/ns#>
PREFIX classif:<http://www.w3.org/ns/org#>

select distinct ?structure ?label ?url ?status ?type
where
{
{{?agregat hal:person <https://data.archives-ouvertes.fr/author/46799>. } UNION
{?agregat hal:person <https://data.archives-ouvertes.fr/author/270747>. } UNION
{?agregat hal:person <https://data.archives-ouvertes.fr/author/yolaine-bourda>. } UNION
{?agregat hal:person <https://data.archives-ouvertes.fr/author/1420714>. } UNION
{?agregat hal:person <https://data.archives-ouvertes.fr/author/4635>. } UNION
{?agregat hal:person <https://data.archives-ouvertes.fr/author/1295368>. }}
?agregat hal:structure ?structure
optional {?structure skos:prefLabel ?label}
optional {?structure vcard:url ?url}
optional {?structure hal:status ?status}
optional {?structure classif:classification  ?type}
}
"""

In [22]:
sparql.setTimeout(60)
sparql.setQuery(query)
sparql.setReturnFormat(CSV)
results2 = sparql.query().convert()
#print (results2)

Pour bien faire, je devrais préciser les liens de subordination entre les structures, mais aussi associer les membres.
Pour faire simple, je vais simplement rattacher author/yolaine-bourda.

In [23]:
tmpResult = results2.decode().split('\n')[1:-1]
with onto:
    for l in tmpResult:
        #print(l)
        structure,label,url,status,sType =l.split(',')
        structure = myStrip(structure)
        label = myStrip(label)
        url = myStrip(url)
        status = myStrip(status)
        sType = myStrip(sType)

        #print('s:',structure,' l:', label,' u:',url,' s:',status,' t:',sType)
        struc = Affiliation(namespace=onto)
        struc.set_iri(structure)
        struc.member = [trueIdHalPerson] # je donne un membre un peu arbitrairement ...
        if label is not None:
            struc.label = label
        if url is not None:
            struc.url = url
        if status is not None:
            struc.status = status
        if sType is not None:
            struc.affiliationType = sType


In [24]:
onto.save("prepaPFR_3.owl")

### II.4 import des documents depuis HAL
On va créer les publications. Je filtre les articles qui ont plus de 30 auteurs car je pense qu'il s'agit de données erronées (cf. fichier joint).

In [25]:
query="""PREFIX dcterms:<http://purl.org/dc/terms/>
PREFIX doc:<https://data.archives-ouvertes.fr/document/>
PREFIX ore:<http://www.openarchives.org/ore/terms/>
PREFIX hal:<http://data.archives-ouvertes.fr/schema/>
PREFIX foaf:<http://xmlns.com/foaf/0.1/>

select  distinct ?document  ?NumCreators
where
{
?document dcterms:hasVersion ?version.
?version dcterms:creator ?creator.

{{?creator hal:person <https://data.archives-ouvertes.fr/author/46799>. } UNION
{?creator hal:person <https://data.archives-ouvertes.fr/author/270747>. } UNION
{?creator hal:person <https://data.archives-ouvertes.fr/author/yolaine-bourda>. } UNION
{?creator hal:person <https://data.archives-ouvertes.fr/author/1420714>. } UNION
{?creator hal:person <https://data.archives-ouvertes.fr/author/4635>. } UNION
{?creator hal:person <https://data.archives-ouvertes.fr/author/1295368>. }}

{
select ?version count(distinct ?o) as ?NumCreators
where {
?version dcterms:creator ?o.
}}
}
group by ?document ?NumCreators
HAVING ( ?NumCreators < 30)
"""

In [26]:
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
tabPublications = extractUriValues(results)
#print(tabPublications)

In [27]:
print(tabPublications)

['https://data.archives-ouvertes.fr/document/hal-01488201', 'https://data.archives-ouvertes.fr/document/hal-00322325', 'https://data.archives-ouvertes.fr/document/hal-00260806', 'https://data.archives-ouvertes.fr/document/hal-00921199', 'https://data.archives-ouvertes.fr/document/hal-00505069', 'https://data.archives-ouvertes.fr/document/hal-01764417', 'https://data.archives-ouvertes.fr/document/hal-00625585', 'https://data.archives-ouvertes.fr/document/hal-01275280', 'https://data.archives-ouvertes.fr/document/hal-00263445', 'https://data.archives-ouvertes.fr/document/hal-01107870', 'https://data.archives-ouvertes.fr/document/hal-00696423', 'https://data.archives-ouvertes.fr/document/hal-00263479', 'https://data.archives-ouvertes.fr/document/hal-00262408', 'https://data.archives-ouvertes.fr/document/hal-01275277', 'https://data.archives-ouvertes.fr/document/hal-01763808', 'https://data.archives-ouvertes.fr/document/hal-01698852', 'https://data.archives-ouvertes.fr/document/hal-0026120

In [28]:
with onto:
    for i in tabPublications:
        #print (i)
        d = foaf.Document(namespace=onto)
        d.set_iri(i)
        # p.isDefinedBy = ["http://sparql.archives-ouvertes.fr/"]
        p = Publication(name=None,namespace=onto,halDocument = d)
        d.maPublicationFromDocument = p

In [29]:
onto.save("prepaPFR_4.owl")

In [30]:
def queryAuthor(doc) :\
    return """PREFIX dcterms:<http://purl.org/dc/terms/>
    PREFIX doc:<https://data.archives-ouvertes.fr/document/>
    PREFIX hal:<http://data.archives-ouvertes.fr/schema/>

    #select ?version ?title ?pdf GROUP_CONCAT(?name,',') AS ?authors
    select distinct ?person
    where
    {<"""+doc+"""> dcterms:hasVersion ?version.
    ?version dcterms:creator ?creator.  #; ore:aggregates ?pdf.
    ?creator hal:person ?person.
    }"""

In [31]:
with onto:
    for i in tabPublications:
            #print(i)
            query_ = queryAuthor(i)
            #print(query)
            sparql.setQuery(query_)
            sparql.setReturnFormat(JSON)
            results_ = sparql.query().convert()
            tabAuteurs = extractUriValues(results_)
            foafDoc = onto.search_one(type = foaf.Document, iri = i)
            #print('fD:',foafDoc)
            #lPubli = foafDoc.maPublication
            # sur une publication, on n'a qu'un seul maPublication
            publi = foafDoc.maPublicationFromDocument
            print("traitement de :",publi,len(tabAuteurs))
            for autIri in tabAuteurs:
                foafAuteur = onto.search_one(type = foaf.Person, iri = autIri)

                if foafAuteur is None: # c'est un coauteur de YB qui n'a pas encore été rencontré
                    #print(\"création d'un auteur\")
                    foafAuteur = foaf.Person()
                    foafAuteur.set_iri(autIri)

                ontoPersonne = foafAuteur.maPersonneFromPerson
                if ontoPersonne is not None:
                    print(ontoPersonne, "ecrit", publi)
                    ontoPersonne.ecrit.append(publi)

                foafAuteur.maPublicationFromPerson.append(publi)
                #print ('fA:',foafAuteur.iri)\n",
                foafAuteur.publications.append(foafDoc)
                foafDoc.monFoafEcritPar.append(foafAuteur)

traitement de : prepaFilRouge.publication1 3
prepaFilRouge.personne7 ecrit prepaFilRouge.publication1
traitement de : prepaFilRouge.publication2 4
prepaFilRouge.personne7 ecrit prepaFilRouge.publication2
traitement de : prepaFilRouge.publication3 3
prepaFilRouge.personne8 ecrit prepaFilRouge.publication3
traitement de : prepaFilRouge.publication4 4
prepaFilRouge.personne8 ecrit prepaFilRouge.publication4
traitement de : prepaFilRouge.publication5 4
prepaFilRouge.personne8 ecrit prepaFilRouge.publication5
traitement de : prepaFilRouge.publication6 4
prepaFilRouge.personne8 ecrit prepaFilRouge.publication6
traitement de : prepaFilRouge.publication7 4
prepaFilRouge.personne8 ecrit prepaFilRouge.publication7
traitement de : prepaFilRouge.publication8 5
prepaFilRouge.personne11 ecrit prepaFilRouge.publication8
traitement de : prepaFilRouge.publication9 2
prepaFilRouge.personne8 ecrit prepaFilRouge.publication9
traitement de : prepaFilRouge.publication10 3
prepaFilRouge.personne8 ecrit prepa

In [32]:
onto.save("prepaPFR_5.owl")

Remarque : il n'y a pas d'article dans HAL ayant comme auteur YB et disposant d'un id arXive. (Dommage, car il y a bien un de vos articles dans HAL.)
Du coup, je ne vais pas plus loin dans l'import des données des publications.

### III Utilisation du raisonneur
### III.1 Parametrage
owlready2.JAVA_EXE=r'C:\Program Files\Java\jre1.8.0_351\bin\jave.exe'
provoque chez moi l'erreur FileNotFoundError. Il s'avère complexe de trouver de quel fichier il s'agit....

owlready2.JAVA_EXE=r'C:\Program Files (x86)\Java\jre1.8.0_261\bin\java.exe'
provoque chez moi l'erreur OwlReadyJavaError: Java error message is:
Error occurred during initialization of VM
Could not reserve enough space for 2048000KB object heap
Cette erreur peut être contournée en effectuant une modification dans le code owlready2.
Il s'agit dans reasoning.py de remplacer JAVA_MEMORY = 2000 par JAVA_MEMORY = 2000 (1500 ne passe pas).
Mais cela n'est pas très propre comme manière de faire.

En téléchargeant Java SE development kit 19.0.1 (JDK), je n'ai plus de problème !


In [33]:
# Change the following PATH if needed
owlready2.JAVA_EXE=r'c:\Program Files\Java\jdk-19\bin\java.exe'
#owlready2.JAVA_EXE='/usr/bin/java'


### III.2 Test simple
Je peux tester une partie de l'ontologie :

In [34]:
sync_reasoner([foaf])

* Owlready2 * Running HermiT...
    c:\Program Files\Java\jdk-19\bin\java.exe -Xmx2000M -cp C:\Users\tof\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\hermit;C:\Users\tof\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\hermit\HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:///C:/Users/tof/AppData/Local/Temp/tmptx5f6kp4
* Owlready2 * HermiT took 1.0948777198791504 seconds
* Owlready * Reparenting foaf.isPrimaryTopicOf: {owl.InverseFunctionalProperty, foaf.page, owl.ObjectProperty} => {foaf.page, owl.InverseFunctionalProperty}
* Owlready * Reparenting foaf.tipjar: {foaf.page, owl.ObjectProperty} => {foaf.page}
* Owlready * Reparenting foaf.weblog: {owl.InverseFunctionalProperty, foaf.page, owl.ObjectProperty} => {foaf.page, owl.InverseFunctionalProperty}
* Owlready * Reparenting foaf.img: {foaf.depiction, owl.ObjectProperty} => {foaf.depiction}
* Owlready * Reparenting foaf.homepage: {foaf.isPrimaryTopicOf, owl.InverseFunctiona

Effectivement il y a quelques soucis mineurs : foaf.img est propriété dérivée de foaf.depiction. C'est donc forcement un ObjectProperty. (Pour être précis, img est une relation entre une foaf.person et une foaf.image qui "which are particularly representative". Je ne retrouve pas img dans protégé.)
La propriété nickname est à la fois une propriété et une data property.


In [35]:
sync_reasoner([foaf])

* Owlready2 * Running HermiT...
    c:\Program Files\Java\jdk-19\bin\java.exe -Xmx2000M -cp C:\Users\tof\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\hermit;C:\Users\tof\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\hermit\HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:///C:/Users/tof/AppData/Local/Temp/tmp7eogclhm
* Owlready2 * HermiT took 0.8590211868286133 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


Après un deuxième appel au raisonneur les anomalies n'apparaissent plus. C'est bien la preuve que les inférences sont prises en compte.

### III.3 Test de l'ontologie

In [36]:
with onto:
    try:
        #sync_reasoner()  # par HermiT
        #sync_reasoner(infer_property_values=True, debug=True, keep_tmp_file=True)
        #sync_reasoner_pellet()
        sync_reasoner_pellet(infer_property_values=True, infer_data_property_values=False, debug=True, keep_tmp_file=True)
        print("Ok, the ontology is consistent.")
    except OwlReadyInconsistentOntologyError:
        print("The ontology is inconsistent!")

* Owlready2 * Running Pellet...
    c:\Program Files\Java\jdk-19\bin\java.exe -Xmx2000M -cp C:\Users\tof\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\antlr-3.2.jar;C:\Users\tof\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\antlr-runtime-3.2.jar;C:\Users\tof\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\aterm-java-1.6.jar;C:\Users\tof\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\commons-codec-1.6.jar;C:\Users\tof\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\httpclient-4.2.3.jar;C:\Users\tof\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\httpcore-4.2.2.jar;C:\Users\tof\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\jcl-over-slf4j-1.6.4.jar;C:\Users\tof\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\jena-arq-2.10.0.jar;C:\Users\tof\AppData\Local\Programs\Python

* Owlready * Adding relation document.hal-00765342 maPublication prepaFilRouge.publication49
* Owlready * Adding relation prepaFilRouge.publication17 monFoafEcritPar prepaFilRouge.personne8
* Owlready * Adding relation prepaFilRouge.publication33 monFoafEcritPar prepaFilRouge.personne8
* Owlready * Adding relation document.hal-00589829 maPublication prepaFilRouge.publication33
* Owlready * Adding relation document.hal-00260806 maPublication prepaFilRouge.publication3
* Owlready * Adding relation author.9988 maPublication prepaFilRouge.publication14
* Owlready * Adding relation author.9988 maPublication prepaFilRouge.publication8
* Owlready * Adding relation prepaFilRouge.publication4 monFoafEcritPar prepaFilRouge.personne8
* Owlready * Adding relation document.hal-00505069 maPublication prepaFilRouge.publication5
* Owlready * Adding relation document.hal-00263491 maPublication prepaFilRouge.publication73
* Owlready * Adding relation document.hal-01763729 maPublication prepaFilRouge.pub

* Owlready2 * Pellet took 1.1977295875549316 seconds
* Owlready2 * Warning: Cannot find new parent '9'
* Owlready2 * Warning: Cannot find new parent '9'
* Owlready2 * Warning: Cannot find new parent '9'
* Owlready2 * Warning: Cannot find new parent '9'
* Owlready * Reparenting prepaFilRouge.Affiliation: {foaf.Organization} => set()
* Owlready * Reparenting structure.303397: {prepaFilRouge.Affiliation} => {prepaFilRouge.Affiliation, foaf.Group}
* Owlready * Reparenting structure.123689: {prepaFilRouge.Affiliation} => {prepaFilRouge.Affiliation, foaf.Group}
* Owlready * Reparenting structure.454174: {prepaFilRouge.Affiliation} => {prepaFilRouge.Affiliation, foaf.Group}
* Owlready * Reparenting structure.2544: {prepaFilRouge.Affiliation} => {prepaFilRouge.Affiliation, foaf.Group}
* Owlready * Reparenting structure.400540: {prepaFilRouge.Affiliation} => {prepaFilRouge.Affiliation, foaf.Group}
* Owlready * Reparenting structure.1050003: {prepaFilRouge.Affiliation} => {prepaFilRouge.Affiliat

En cherchant dans la doc, storid (store id) est une abréviation pour l'iri. On peut obtenir l'iri par :

In [37]:
default_world._unabbreviate(9)

'http://www.w3.org/2000/01/rdf-schema#subClassOf'

In [38]:
with onto:
    try:
        sync_reasoner()  # par HermiT
        #sync_reasoner(infer_property_values=True, debug=True, keep_tmp_file=True)
        #sync_reasoner_pellet()
        #sync_reasoner_pellet(infer_property_values=True, infer_data_property_values=True, debug=True, keep_tmp_file=True)
        print("Ok, the ontology is consistent.")
    except OwlReadyInconsistentOntologyError:
        print("The ontology is inconsistent!")

* Owlready2 * Running HermiT...
    c:\Program Files\Java\jdk-19\bin\java.exe -Xmx2000M -cp C:\Users\tof\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\hermit;C:\Users\tof\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\hermit\HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:///C:/Users/tof/AppData/Local/Temp/tmpxdfhxqko


Ok, the ontology is consistent.


* Owlready2 * HermiT took 0.8330135345458984 seconds
* Owlready2 * Warning: Cannot find new parent '9'
* Owlready2 * Warning: Cannot find new parent '9'
* Owlready2 * Warning: Cannot find new parent '9'
* Owlready2 * Warning: Cannot find new parent '9'
* Owlready * Reparenting prepaFilRouge.Test: {owl.Thing} => set()
* Owlready * Reparenting prepaFilRouge.Publication: {owl.Thing} => set()
* Owlready * Reparenting prepaFilRouge.Personne: {owl.Thing} => set()
* Owlready * Reparenting prepaFilRouge.Affiliation: {owl.Thing} => set()
* Owlready * Reparenting foaf.primaryTopic: {owl.FunctionalProperty, owl.ObjectProperty} => {foaf.topic, owl.FunctionalProperty}
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


HermiT ne trouve rien de plus.

In [39]:
onto.save("prepaPFR_6.owl")

### III.4 Remarque

J'ai désactivé infer_data_property_values car cela provoque une corruption de mon ontologie.
Le fichier produit ne peut plus être ouvert dans protégé.
En lisant le fichier, j'ai un problème : https://data.archives-ouvertes.fr/structure/303397.
Son label pose un problème. C'est un string qui contient un '. Je dois encore investiguer ce point.
(Peut être que le problème est lié au réemploi d'iri structure de HAL sur un classe ayant un autre nom.)

### III.5 test sur PropertyChain

In [40]:
with onto:
    # je n'arrive pas à donner à une ObjectProperty deux caractéristiques
    # je peux passer par une subproperties
    class baseIrreflexiveProperty (ObjectProperty, IrreflexiveProperty):
        domaine = [foaf.Person]
        range = [foaf.Person]

    class coAuteur(ObjectProperty):
        domaine = [foaf.Person]
        range = [foaf.Person]
    coAuteur.property_chain.append(PropertyChain([foaf.publications, monFoafEcritPar]))
    coAuteur.comment = ["Relation entre foaf.Person ayant en commun un foaf.Document"]

In [41]:
onto.save("prepaPFR_6b.owl")

In [42]:
with onto:
    try:
        #sync_reasoner()  # par HermiT
        #sync_reasoner(infer_property_values=True, debug=True, keep_tmp_file=True)
        #sync_reasoner_pellet()
        sync_reasoner_pellet(infer_property_values=True, infer_data_property_values=False, debug=True, keep_tmp_file=True)
        print("Ok, the ontology is consistent.")
    except OwlReadyInconsistentOntologyError:
        print("The ontology is inconsistent!")

* Owlready2 * Running Pellet...
    c:\Program Files\Java\jdk-19\bin\java.exe -Xmx2000M -cp C:\Users\tof\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\antlr-3.2.jar;C:\Users\tof\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\antlr-runtime-3.2.jar;C:\Users\tof\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\aterm-java-1.6.jar;C:\Users\tof\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\commons-codec-1.6.jar;C:\Users\tof\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\httpclient-4.2.3.jar;C:\Users\tof\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\httpcore-4.2.2.jar;C:\Users\tof\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\jcl-over-slf4j-1.6.4.jar;C:\Users\tof\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\jena-arq-2.10.0.jar;C:\Users\tof\AppData\Local\Programs\Python

* Owlready * Adding relation author.9988 coAuteur author.972490
* Owlready * Adding relation author.9988 coAuteur author.6988
* Owlready * Adding relation author.9988 coAuteur author.1118260
* Owlready * Adding relation author.9988 coAuteur author.9988
* Owlready * Adding relation author.9988 coAuteur author.4635
* Owlready * Adding relation author.1118260 coAuteur author.972490
* Owlready * Adding relation author.1118260 coAuteur author.6988
* Owlready * Adding relation author.1118260 coAuteur author.1118260
* Owlready * Adding relation author.1118260 coAuteur author.9988
* Owlready * Adding relation author.1118260 coAuteur author.4635
* Owlready * Adding relation author.352970 coAuteur author.352971
* Owlready * Adding relation author.352970 coAuteur author.71457
* Owlready * Adding relation author.352970 coAuteur author.46799
* Owlready * Adding relation author.352970 coAuteur author.352970
* Owlready * Adding relation author.11209049 coAuteur author.11209049
* Owlready * Adding rel

* Owlready2 * Pellet took 0.9625461101531982 seconds
* Owlready2 * Warning: Cannot find new parent '9'
* Owlready2 * Warning: Cannot find new parent '9'
* Owlready2 * Warning: Cannot find new parent '9'
* Owlready2 * Warning: Cannot find new parent '9'
* Owlready * Reparenting prepaFilRouge.Affiliation: {owl.Thing} => set()
* Owlready * Reparenting prepaFilRouge.Personne: {owl.Thing} => set()
* Owlready * Reparenting prepaFilRouge.Publication: {owl.Thing} => set()
* Owlready * Reparenting prepaFilRouge.Test: {owl.Thing} => set()
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [43]:
onto.save("prepaPFR_7.owl")

J'arrive à créer la relation coAuteur, mais je n'arrive pas à la rendre irreflexive.